## Day 47 Proyect: Amazon Price Tracker Email

### Proyecto Día 47 – Alerta de Precio desde Amazon con Email 📉📬

Este proyecto automatiza la monitorización del precio de un producto en Amazon.  
Si el precio baja por debajo de un umbral predefinido, se envía una alerta por correo electrónico al usuario.

🔧 Herramientas aplicadas:
- Web scraping con `requests` y `BeautifulSoup`
- Headers personalizados para evitar bloqueo por bots
- Envío de correos con `smtplib` usando contraseña de aplicación


## Paso 1 – Scraping del precio y nombre del producto

Obtenemos el HTML del producto en Amazon, localizamos el título y el precio.


In [1]:
import requests                             # Libreria para hacer peticiones HTTP
from bs4 import BeautifulSoup               # Libreria para parsear/extraer datos de HTML
import smtplib                              # Libreria para enviar correos
import os                                   # Libreria para interactuar con el sistema operativo
from dotenv import load_dotenv              # Libreria para cargar variables de entorno desde un archivo .env
from email.mime.text import MIMEText        # Libreria para cuerpos de correos en texto plano
from email.header import Header             # Libreria para codificar encabezados de correos

In [2]:
# Carga las variables del archivo .env
load_dotenv()  

True

In [3]:
# URL del producto a monitorear
URL = "https://www.amazon.com.mx/dp/B085WBNFTP/"

# Headers para simular navegador  (esto es útil para evitar bloqueos por parte del servidor)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept-Language": "es-ES,es;q=0.9,en;q=0.8"
}

# Realiza la solicitud GET al endpoint de la página web
response = requests.get(URL, headers=headers)

# Crea un objeto BeautifulSoup para parsear el HTML
soup = BeautifulSoup(response.text, "html.parser")

#print(soup.prettify())  # Imprime el HTML formateado para ver la estructura


In [4]:
# Extraer el título usando el selector div 
title_element = soup.find('div', class_='_c3AtZ_title-component-overflow3_3SMef')  # Busca el elemento del título
product_title = (title_element 
                .get_text()                                         # Extrae el texto
                .split(":")[1]                                      # Divide el texto por ":" y toma el segundo elemento
                .strip()                                            # Elimina espacios en blanco al inicio y al final
                ) if title_element else "Título no encontrado"      # Si no se encuentra, asigna un valor por defecto

# Extraer el precio usando el selector span
price_element = soup.find("span", class_="a-price-whole")       # Busca el elemento del precio
product_price = (price_element
                .get_text()                                     # Extrae el texto
                .replace(",", "")                               # Reemplaza las comas por nada
                .replace(".", "")                               # Reemplaza los puntos por nada
                ) if price_element else "Precio no encontrado"  # Si no se encuentra, asigna un valor por defecto

# Imprime el título y el precio del producto
print(f"Título del producto: {product_title}")
print(f"Precio del producto: {product_price}")


Título del producto: Birdman Falcon Performance Proteina Premium En Polvo, 30gr proteina, 3gr Creatina, 45 Porciones Sabor Golden Vanilla 1.9Kg
Precio del producto: 1331


#### Paso 2 – Comparar precio con umbral y enviar email si es necesario

Establecemos un precio objetivo y enviamos un correo si el precio actual está por debajo de ese umbral.


In [5]:
# Datos de autenticación 
EMAIL_SENDER_MAIL = os.environ["EMAIL_SENDER_MAIL"]
EMAIL_APP_PASS = os.environ["EMAIL_APP_PASS"]
EMAIL_RECEIVER_MAIL = os.environ["EMAIL_RECEIVER_MAIL"]

In [6]:
# Umbral de alerta
PRECIO_OBJETIVO = 1500

In [7]:
# Crear el cuerpo del mensaje
mail_body = f"""
📦 Producto: {product_title}

💰 Precio actual: ${product_price}

🛒 Compra aquí: {URL}
"""

In [8]:
# Verificar si el precio es menor que el precio objetivo
if int(product_price) < PRECIO_OBJETIVO:
    # Si el precio es menor al objetivo, se envía un correo
    
    print("✅ Precio por debajo del objetivo. Se enviará correo.")

    # Crear el objeto MIMEText con charset utf-8
    msg = MIMEText(mail_body, _subtype="plain", _charset="utf-8")

    #  Asignar cabeceras 
    msg['From']    = EMAIL_SENDER_MAIL
    msg['To']      = EMAIL_RECEIVER_MAIL
    msg['Subject'] = f"🔔 Alerta de precio: {product_title[:52]}..."

    # Enviar correo usando SMTPlib
    with smtplib.SMTP("smtp.gmail.com", port=587) as connection:
        connection.starttls()                                               # Iniciar conexión segura
        connection.login(user=EMAIL_SENDER_MAIL, password=EMAIL_APP_PASS)   # Iniciar sesión con tus credenciales
        connection.send_message(msg)                                        # Enviar el mensaje
        print(f"✅ Correo enviado a {EMAIL_RECEIVER_MAIL}")

else:
    # Si el precio no es menor al objetivo, no se envía correo
    print("⏳ Aún no baja del precio deseado. No se envía notificación.")

✅ Precio por debajo del objetivo. Se enviará correo.
✅ Correo enviado a riveraderrick5@gmail.com


#### 🧠 Conclusión

Este proyecto pone en práctica habilidades valiosas:

- 💡 Scraping en sitios reales con estructura dinámica
- 💻 Simulación de navegación real mediante headers HTTP
- 📧 Automatización de alertas vía email con `smtplib`
- 🔐 Manejo seguro de credenciales con variables de entorno